# Chapter 11 - Visual Odometry / Visual SLAM

In this chapter we go over Structure from Motion in more detail. First we will look the the case where we want to construct a 3d scene from many not sequetially ordered images. Then we will traverse towards the case where we have sequetial ordered images. Towards the end of this chapter we will focus on the problem of loopclosure and itroduce therefore the SLAM algorithm.


## Multi-view SFM

In structure from motion we want to recover the 3D structure of a scene from many differen images of a moving camera. However the images do not always have to be in a squetial order. Sometimes we also want to construct a scene from a set of images of the same scene from different alges, cameras and even from different days. In this case we can not rely on the sequetial order and therefore we do not know the spacial and temporal relation between different images. In such a setting we have to apply the **Hierarchical SFM**.



### Hierarchical SFM

The name for this methods is based on the hierarchical structure in which the images are ordered to give introduce some kind of relation between the images. In the following we will how we can creat such a structure.

First we have to extract features from the images and then we have to search the orther images for images which have matching features. We match two images together if they have matching images/frames, we call them nearby frames. The we spilt the image set into clusters of 3 nearby/matching frames. Now we can use two of the images to apply SFM. As a result we get the translation and rotation between the cameras. From this informtaion we can now triangluate the points int he 3D space and create a point cloud. We can then merge the third point into the pointcloud by using the 3-point RANSAC between pointcloud and third image. we get again the rotation and translation of the third view in respect to a reference view of the pointcloud. Refer the the section about 3D-to-2D later on in this chapter. Like this we then can add new features of the third view to the point cloud. We can now merge clusters pairwisse using matching features in both clusters.this we can do as before using the roation and translation between both reference views. This can be done using RANSAC again. In the end we can use Bundle Adjustment to refine the tructure as well as camera locations.

![Hierarchical SFM](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_11/1_hierarchical_sfm.png)
*Figure 1: Hierarchical SFM. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*


### Sequential SFM aka Visual Odometry

In sequential SFM also called **Visual Odometry** we have sequetial images. Also Visual Odometry comuptes the path of the camera sequetially, so pose after pose. This aloows for real applications. The general flow of a VO algorithm can be split into two parts, the front-end and the back-end. The front-end is responsible for the feature detection and amtching and the pose estimations. In the backend the pose estimations the the locally optimized.

![VO Flow Chart](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_11/2_vo_flow.png)
*Figure 2: VO Flow Chart. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

Most of the front-end tasks we have already seen in previous chapter. We will not repeate the feture detection and matching here. However we will mention some things about the motion estimation, which also has already been discussed in chapter XY. As it can be seen in Figure 3 there are different motion estimation settings which we will now briefly explain.


#### 2D-to-2D / Motion from Image Feature Correspondences

In this setting we have to feature points $f_{k-1}$ and $f_{k}$ where $k$ denotes the frame number. Both of these feature points are specified in 2D so on the image plane. Using the 5-point algorithm we can get a solution, therefore we need at least 5 point correspondences. In gereral the more the better so we can also use the 8-point algorithm. The solution is then found by minimizing the reprojection error. 

![2D-to-2D](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_11/3_2d_to_2d.png)
*Figure 3: 2D-to-2D. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*


#### 3D-to-2D / Motion from 3D Structure and Image Correspondences

In such a case we have the feature $f_{k-1}$ in 3D and $f_k$ in 2D. So in other words we have a point cloud and a image. We know this problem as the *camera from resection* or more familiar as PnP problem. From chapter 4 we know that we need at least 3+1 correspondences to get a unique solution. We can use the P3P algorithm to find a solution.

![3D-to-2D](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_11/4_3d_to_2d.png)
*Figure 4: 3D-to-2D. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*


#### 3D-to-3D / Motion from 3D Structure and Image Correspondences

In this last setting we have for both features $f_{k-1}$ and $f_k$ the 3D points. To achieve this we need to have triangulated the points first using a stereo camera for example. In this case we need at least 3 non-colliniear correspondences. The solution is then found by minimizing the 3D-3D euclidean distance. The popular algorithm in this case is Arun87 (Least-Squares Fitting of Two 3-D Point Sets) for gloabal registraion with bundle adjustment.

![3D-to-3D](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_11/5_3d_to_3d.png)
*Figure 5: 3D-to-3D. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*


We can sort the above noted motion estimation intoo monocular and stereo vision as following:

| Type of correspondences        | Monocular     | Stereo  |
| ------------------------------ |:-------------:| -------:|
| 2D-2D                          | X             | -       |
| 3D-2D                          | X             | X       |
| 3D-3D                          | -             | X       |

### Molocular VO Example

To start of we have to initialize the algorithm with a pointcloud. So we start of with a 2D-to-2D case. For the initialisation we need two frames that overlap in feature points however thes also should not overlap to much as otherwise the point clound would not be big enought. As seen before we can use 5 or 8 point algorithhm for this initialisation. after that we can use bundle adjustment to refine the pointcloud and the pose of the two frames. The to frames we use for initalisation we call are also keyframes. The whole initialsation process is called **bootstrapping**.

![Bootstrapping](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_11/6_bootstrapping.png)
*Figure 6: Bootstrapping. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

So one of the key questions is how far apart do the images have to be to achieve good initialisation. If they are to close together, so when the baseline is too small, then there is a large depth uncertainty. If they are far apart then the depth is mor certain. However too far apart the images should also not be because otherwise there are not anought overlapping features anymore. As a result we skip some of the frame to have frames that are not to cloe to each others. We skipp untill the uncertainty falls below a threshold. The resulting selected frames are called **keyframes**. As a rule of thumb we add a ekyframe if the ratio of keframe distance devided by the average depth is larger than 10-20%.

![skipping frames](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_11/7_skipping_keyframes.png)
*Figure 7: skipping frames. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

From the pointcloud prodiced by the first two keyframes we can calculate the pose for every subsequent frame using the P3P or DLT algorithm as we are now in a PnP or 3D-to-2D setting. We can get the pose as long as there are still enough point correspondencces between the existing pointcloud and the new frames. Depending on the algorithms we need at least 6 (DLT) or 3+1 (P3P) points. 

![Localisation](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_11/8_localisation.png)
*Figure 8: Localisation. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*


When there are not enought correspondencs anymore, then we have to set a new keyframe. By adding a new Keyframe we can add new points to the pointcloud using triangulation. Now that we have new point in the pointcloud we can get the pose for the next few frames without adding new 3D points. However since the pointcloud is already initialized we would also be able to add every frame as key frame from now on. **>>>Can we find More Pro/Cons here...<<<**

![Mapping](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_11/9_mapping.png)
*Figure 9: Mapping. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*
